In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dino_functions import Dinov2

/home/nirshal/miniconda3/envs/primary/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/nirshal/miniconda3/envs/primary/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [29]:
grasp_path = "resources/pouch3/grasp_pose_pouch3.npy"
gt_grasp_path = "resources/pouch3/gt_grasp_pose_pouch3.npy"

matrix = np.load(grasp_path, allow_pickle=True).item()

matrix[0][0:3,0] = -matrix[0][0:3,0]
matrix[0][0:3,2] = -matrix[0][0:3,2]

np.save(grasp_path, matrix)

In [30]:
gt_grasp_axes = np.load(grasp_path, allow_pickle=True).item()
grasp_axes = np.load(gt_grasp_path, allow_pickle=True).item()

center_error = {}
for key in gt_grasp_axes:
    center_gt = gt_grasp_axes[key][0:3,3]
    center_pred = grasp_axes[key][0:3,3]
    center_error[key] = np.linalg.norm(center_gt - center_pred)

# Error Between the x axis
approach_axis_error = {}
for key in gt_grasp_axes:
    gt_x = gt_grasp_axes[key][0:3,0]
    pred_x = grasp_axes[key][0:3,0]
    # approach_axis_error[key] = np.arccos(np.dot(gt_x, pred_x)/(np.linalg.norm(gt_x)*np.linalg.norm(pred_x)))
    approach_axis_error[key] = np.arccos(np.dot(gt_x, pred_x))
    # convert to degrees
    approach_axis_error[key] = np.degrees(approach_axis_error[key])

print("Center Error: ", center_error)
print("Approach Axis Error: ", approach_axis_error)

Center Error:  {0: 0.016436363856055725}
Approach Axis Error:  {0: 5.888322534993989}


In [1]:
import open3d as o3d
# draw a sphere
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
mesh_sphere.compute_vertex_normals()
mesh_sphere.paint_uniform_color([0.1, 0.1, 0.7])
# vizualize the sphere
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(mesh_sphere)
vis.run()
vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


: 